In [1]:
#Example 16.4
#Import data, define sets and dictionary
Data = readcsv("Eg_16.4.csv");
S = ["HP"];
W = ["CW"];
K = length(Data[1,:]) -1;
Hk = [["HP"],["H1"],["H1","H2"],["H1","H2"]];
Hk2 = [["HP"],["HP","H1"],["HP","H1","H2"],["HP","H1","H2"]];
Ck = [["C1"],["C1"],["C1","C2"],["C2","CW"]];
H = ["H1","H2","HP"];
C = ["C1","C2","CW"];
Q = Dict(("H1",1) => 0.1)
for i in 1:length(Data[:,1])
    for j in 1:K
        Q[(Data[i,1],j)] = Data[i,j+1];
    end
end

In [2]:
#Solution 1, on page 543
using JuMP
using Gurobi
Ma = Model(solver = GurobiSolver(OutputFlag = 0));
@variable(Ma, Qkij[k=1:K,i in H,j in C] >= 0);
@variable(Ma, Rki[k=0:K,i in H] >= 0);
@variable(Ma, y[H,C,["a","b"]], Bin);
@objective(Ma, Min, sum(y));
@constraint(Ma, [i in H], Rki[0,i] == 0); 
@constraint(Ma, [i in H], Rki[K,i] == 0); 
@constraint(Ma, [k in 1:K, i in Hk2[k]], Rki[k,i] - Rki[k-1,i] + sum(Qkij[k,i,j] for j in Ck[k]) == Q[i,k]);
@constraint(Ma, [k in 1:K, j in Ck[k]], sum(Qkij[k,i,j] for i in Hk2[k]) == Q[j,k]);
@constraint(Ma, [i in H, j in C], sum(Qkij[k,i,j] for k in 1:2) - 10000*y[i,j,"a"] <= 0);
@constraint(Ma, [i in H, j in C], sum(Qkij[k,i,j] for k in 3:4) - 10000*y[i,j,"b"] <= 0);
# To force extra variables to be zero. This is optional in this example.
for k in 1:K
   for ii in H
        for jj in C
        if in(ii,Hk2[k]) == 0 || in(jj,Ck[k]) == 0
          @constraint(Ma, Qkij[k,ii,jj] == 0);
        end
        end
    end
    for ii in H
        if in(ii,Hk2[k]) == 0
          @constraint(Ma, Rki[k,ii] == 0);
        end
    end
end
@constraint(Ma, Qkij[4,"H2","CW"] >= 220); # To get the same solution as book
solve(Ma);
println("Above pinch:");
for i in H
    for j in C
        if getvalue(y[i,j,"a"]) >= 0.1
            println("Match ",i," and ",j,": ")
            for k in 1:2
            if getvalue(Qkij[k,i,j]) >= 0.1
                println("    Q\("i,",",j,",",k,"\) = ",getvalue(Qkij[k,i,j]));
            end
            end
        end        
    end
end
println("Below pinch:");
for i in H
    for j in C
        if getvalue(y[i,j,"b"]) >= 0.1
            println("Match ",i," and ",j,": ")
            for k in 3:4
            if getvalue(Qkij[k,i,j]) >= 0.1
                println("    Q\("i,",",j,",",k,"\) = ",getvalue(Qkij[k,i,j]));
            end
            end
        end        
    end
end

Academic license - for non-commercial use only
Above pinch:
Match H1 and C1: 
    Q(H1,C1,2) = 60.0
Match HP and C1: 
    Q(HP,C1,1) = 30.0
    Q(HP,C1,2) = 30.0
Below pinch:
Match H1 and C1: 
    Q(H1,C1,3) = 24.999999999999957
Match H1 and C2: 
    Q(H1,C2,3) = 117.00000000000003
    Q(H1,C2,4) = 78.00000000000001
Match H2 and C1: 
    Q(H2,C1,3) = 215.00000000000006
Match H2 and CW: 
    Q(H2,CW,4) = 225.0


In [3]:
#Solution 2, on page 545
Mb = Model(solver = GurobiSolver(OutputFlag = 0));
@variable(Mb, Qkij[k=1:K,i in H,j in C] >= 0);
@variable(Mb, Rki[k=0:K,i in H] >= 0);
@variable(Mb, y[H,C,["a","b"]], Bin);
@objective(Mb, Min, sum(y));
@constraint(Mb, [i in H], Rki[0,i] == 0);
@constraint(Mb, [i in H], Rki[K,i] == 0);  
@constraint(Mb, [k in 1:K, i in Hk2[k]], Rki[k,i] - Rki[k-1,i] + sum(Qkij[k,i,j] for j in Ck[k]) == Q[i,k]);
@constraint(Mb, [k in 1:K, j in Ck[k]], sum(Qkij[k,i,j] for i in Hk2[k]) == Q[j,k]);
@constraint(Mb, [i in H, j in C], sum(Qkij[k,i,j] for k in 1:2) - 10000*y[i,j,"a"] <= 0);
@constraint(Mb, [i in H, j in C], sum(Qkij[k,i,j] for k in 3:K) - 10000*y[i,j,"b"] <= 0);
@constraint(Mb, sum(y["H1","C1","b"]) == 0);
for k in 1:K
   for ii in H
        for jj in C
        if in(ii,Hk2[k]) == 0 || in(jj,Ck[k]) == 0
          @constraint(Mb, Qkij[k,ii,jj] == 0);
        end
        end
    end
    for ii in H
        if in(ii,Hk2[k]) == 0
          @constraint(Mb, Rki[k,ii] == 0);
        end
    end
end
solve(Mb);
println("Above pinch:");
for i in H
    for j in C
        if getvalue(y[i,j,"a"]) >= 0.1
            println("Match ",i," and ",j,": ")
            for k in 1:2
            if getvalue(Qkij[k,i,j]) >= 0.1
                println("    Q\("i,",",j,",",k,"\) = ",getvalue(Qkij[k,i,j]));
            end
            end
        end        
    end
end
println("Below pinch:");
for i in H
    for j in C
        if getvalue(y[i,j,"b"]) >= 0.1
            println("Match ",i," and ",j,": ")
            for k in 3:4
            if getvalue(Qkij[k,i,j]) >= 0.1
                println("    Q\("i,",",j,",",k,"\) = ",getvalue(Qkij[k,i,j]));
            end
            end
        end        
    end
end

Academic license - for non-commercial use only
Above pinch:
Match H1 and C1: 
    Q(H1,C1,2) = 60.0
Match HP and C1: 
    Q(HP,C1,1) = 30.0
    Q(HP,C1,2) = 30.0
Below pinch:
Match H1 and C2: 
    Q(H1,C2,3) = 117.0
    Q(H1,C2,4) = 78.00000000000001
Match H1 and CW: 
    Q(H1,CW,4) = 24.999999999999403
Match H2 and C1: 
    Q(H2,C1,3) = 240.0
Match H2 and CW: 
    Q(H2,CW,4) = 200.0000000000006


In [4]:
#Solution 3 - no pinch partition, on page 545-546
Mc = Model(solver = GurobiSolver(OutputFlag = 0));
@variable(Mc, Qkij[k=1:K,i in H,j in C] >= 0);
@variable(Mc, Rki[k=0:K,i in H] >= 0);
@variable(Mc, y[H,C], Bin);
@objective(Mc, Min, sum(y));
@constraint(Mc, [i in H], Rki[2,i] == 0);
@constraint(Mc, [i in H], Rki[K,i] == 0);  
@constraint(Mc, [k in 1:K, i in Hk2[k]], Rki[k,i] - Rki[k-1,i] + sum(Qkij[k,i,j] for j in Ck[k]) == Q[i,k]);
@constraint(Mc, [k in 1:K, j in Ck[k]], sum(Qkij[k,i,j] for i in Hk2[k]) == Q[j,k]);
@constraint(Mc, [i in H, j in C], sum(Qkij[k,i,j] for k in 1:K) - 10000*y[i,j] <= 0);
for k in 1:K
   for ii in H
        for jj in C
        if in(ii,Hk2[k]) == 0 || in(jj,Ck[k]) == 0
          @constraint(Mc, Qkij[k,ii,jj] == 0);
        end
        end
    end
    for ii in H
        if in(ii,Hk2[k]) == 0
          @constraint(Mc, Rki[k,ii] == 0);
        end
    end
end
solve(Mc);
for i in H
    for j in C
        if getvalue(y[i,j]) >= 0.1
            println("Match ",i," and ",j,": ")
            for k in 1:4
            if getvalue(Qkij[k,i,j]) >= 0.1
                println("    Q\("i,",",j,",",k,"\) = ",getvalue(Qkij[k,i,j]));
            end
            end
        end        
    end
end

Academic license - for non-commercial use only
Match H1 and C1: 
    Q(H1,C1,2) = 60.0
    Q(H1,C1,3) = 24.999999999999915
Match H1 and C2: 
    Q(H1,C2,3) = 117.0
    Q(H1,C2,4) = 78.0
Match H2 and C1: 
    Q(H2,C1,3) = 215.00000000000009
Match H2 and CW: 
    Q(H2,CW,4) = 225.0
Match HP and C1: 
    Q(HP,C1,1) = 30.0
    Q(HP,C1,2) = 30.0
